# Imports

In [2]:
import os
import pandas as pd
from flow.envs import TestEnv

from flow.core.experiment import Experiment

from flow.core.params import VehicleParams
from flow.core.params import NetParams, SumoCarFollowingParams
from flow.core.params import InitialConfig
from flow.core.params import EnvParams
from flow.core.params import SumoParams
from flow.controllers import IDMController, ContinuousRouter
from flow.controllers import RLController
from flow.networks import Network
from flow.core.params import InFlows
from flow.envs import WaveAttenuationPOEnv

from flow.envs import Env
from gym.spaces.box import Box
from gym.spaces import Tuple
import numpy as np

from flow.core.experiment import Experiment
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS


# Network configuration: 

Number of vehicles for initial config, number of vehicles in flow 

In [3]:
vehicles = VehicleParams()

#Human driving vehicles (initially 3)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
            color="white")

#flow of 3000 vehicles per hour for the segment studied (aller)
inflow = InFlows()
inflow.add(
   veh_type = "human",
   edge="486885259",
   vehs_per_hour=3000,
    depart_lane="random",
    depart_speed = "max",
    begin = 1,
    color="green")

inflow.add(
   veh_type = "human",
   edge="154190805",
   vehs_per_hour=3000,
   depart_lane="random",
   depart_speed = "max",
   begin = 1,
   color="green")

#Edges (road segments)
EDGES_DISTRIBUTION = [
    "486885259",
    "154190805",
]

#Road segments positions
class MarocNetwork(Network):

    def specify_routes(self, net_params1):
        return {
            "486885259": [
            "486885259",],

            "154190805" : [
            "154190805",],}


In [4]:
from flow.controllers import RLController
vehicles.add("rl",
             acceleration_controller=(RLController, {}),
             routing_controller=(ContinuousRouter, {}),
             num_vehicles=1)

# RL agent configuration

Action space, states, reward computation

In [4]:
#function action_space
#2 actions possibles pour chaque véhicule RL : +1 acceleration ou -1 acceleration
    
class MarocEnv(Env):
    pass
class MarocEnv(MarocEnv): # update my environment class

    @property
    def action_space(self):
        num_actions = self.initial_vehicles.num_rl_vehicles
        accel_ub = self.env_params.additional_params["max_accel"]
        accel_lb = - abs(self.env_params.additional_params["max_decel"])

        return Box(low=accel_lb,
                   high=accel_ub,
                   shape=(num_actions,))


class MarocEnv(MarocEnv):  # update my environment class

    @property
    def observation_space(self):
        return Box(
            low=0,
            high=float("inf"),
            shape=(2*self.initial_vehicles.num_vehicles,))


class MarocEnv(MarocEnv):  # update my environment class

    def _apply_rl_actions(self, rl_actions):
        # the names of all autonomous (RL) vehicles in the network
        rl_ids = self.k.vehicle.get_ids()

        # use the base environment method to convert actions into accelerations for the rl vehicles
        self.k.vehicle.apply_acceleration(rl_ids, rl_actions)


class MarocEnv(MarocEnv):  # update my environment class

    def get_state(self, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we use the get_absolute_position method to get the positions of all vehicles
        pos = [self.k.vehicle.get_x_by_id(veh_id) for veh_id in ids]

        # we use the get_speed method to get the velocities of all vehicles
        vel = [self.k.vehicle.get_speed(veh_id) for veh_id in ids]

        # the speeds and positions are concatenated to produce the state
        return np.concatenate((pos, vel))



class MarocEnv(MarocEnv):  # update my environment class

    def compute_reward(self, rl_actions, **kwargs):
        # the get_ids() method is used to get the names of all vehicles in the network
        ids = self.k.vehicle.get_ids()

        # we next get a list of the speeds of all vehicles in the network
        speeds = self.k.vehicle.get_speed(ids)

        # finally, we return the average of all these speeds as the reward
        return np.mean(speeds)

# Simulation

In [7]:
#Number of steps
HORIZON = 1000
#Max acceleration/ déceleration (RL Agent)
ADDITIONAL_ENV_PARAMS = {"max_accel": 1, "max_decel": 1,# bounds on the ranges of ring road lengths the autonomous vehicle 
        # is trained on
        "ring_length": [220, 270]}

# SUMO PARAMS: sim_step (temps entre les steps en secondes), render (afficher sur SUMO si True)
sumo_params = SumoParams(sim_step=0.5, render=True, restart_instance=False, emission_path='data')

# ENVIRONMENT PARAMS
env_params = EnvParams(additional_params=ADDITIONAL_ENV_PARAMS, horizon=HORIZON)

# NETWORK PARAMS
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
#33 m/s = 120 Km/s
additional_net_params['speed_limit'] = 33
additional_net_params['highway_lanes'] = 3

#absolute Path to network file 
abs_path = 'C://PHD/VSL/RL-VSL/Project SUMO-VSL/flow-master/tutorials/networks/maroc.net.xml'
net_params1 = NetParams(inflows=inflow, 
                        additional_params=additional_net_params, template=abs_path)
# NETWORK
network = MarocNetwork

# INITIAL CONFIG
initial_config = InitialConfig(edges_distribution=EDGES_DISTRIBUTION) 

# dictionnaire FLOW_PARAM
flow_params = dict( exp_tag  = 'agent_VSL',
                    env_name = MarocEnv,  
                    network  = network,
                    simulator= 'traci',
                    sim      = sumo_params,
                    env      = env_params,
                    net      = net_params1,
                    veh      = vehicles,
                    initial  = initial_config)

# create EXPERIMENT with class created
exp = Experiment(flow_params)
emission_location = os.path.join(exp.env.sim_params.emission_path, exp.env.network.name)

# RUN SIMULATION SUMO
_ = exp.run(1, convert_to_csv=True)
pd.read_csv(emission_location + '-0_emission.csv')

**********************************************************
**********************************************************
**********************************************************
significantly decrease after large number of rollouts. In 
order to avoid this, set SumoParams(restart_instance=True).
**********************************************************
**********************************************************
**********************************************************
Round 0, return: 28171.967530655726
data\agent_VSL_20211103-2316561635977816.465341-0_emission.csv data
Average, std returns: 28171.967530655726, 0.0
Average, std velocities: 28.171967530655706, 0.0
Average, std outflows: 0.0, 0.0
Total time: 269.77622652053833
steps/second: 15.578997568302253


,time,id,x,y,speed,headway,leader_id,target_accel_with_noise_with_failsafe,target_accel_no_noise_no_failsafe,target_accel_with_noise_no_failsafe,target_accel_no_noise_with_failsafe,realized_accel,road_grade,edge_id,lane_number,distance,relative_position,follower_id,leader_rel_speed
0,0.5,flow_00.0,1.793007,5.748684,30.000000,1000.000000,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0,486885259,2,0.000000,5.100000,NaN,-1031.000000
1,1.0,flow_00.0,13.184205,15.509521,30.000000,1000.000000,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.238600e-09,0,486885259,2,15.000000,20.100000,NaN,-1031.000000
2,1.5,flow_00.0,24.575402,25.270357,30.000000,1000.000000,NaN,1.492402e-10,1.492402e-10,1.492402e-10,1.492402e-10,-1.867392e-09,0,486885259,2,30.000000,35.100000,NaN,-1031.000000
3,2.0,flow_00.0,35.930466,35.070843,30.000000,1000.000000,NaN,2.737326e-10,2.737326e-10,2.737326e-10,2.737326e-10,-1.557737e-09,0,486885259,2,45.000000,50.100000,flow_00.1,-1031.000000
4,2.5,flow_00.0,46.704197,45.509235,30.000000,1000.000000,NaN,3.775820e-10,3.775820e-10,3.775820e-10,3.775820e-10,-1.299426e-09,0,486885259,2,60.000000,65.100000,flow_00.1,-1031.000000
5,3.0,flow_00.0,57.477928,55.947627,30.000000,1000.000000,NaN,4.642104e-10,4.642104e-10,4.642104e-10,4.642104e-10,-1.083954e-09,0,486885259,2,75.000000,80.100000,flow_00.1,-1031.000000
6,3.5,flow_00.0,68.400875,66.227411,30.000000,1000.000000,NaN,5.364740e-10,5.364740e-10,5.364740e-10,5.364740e-10,-9.042083e-10,0,486885259,2,90.000000,95.100000,flow_00.1,-1031.000000
7,4.0,flow_00.0,79.476694,76.344700,30.000000,1000.000000,NaN,5.967546e-10,5.967546e-10,5.967546e-10,5.967546e-10,-7.542695e-10,0,486885259,2,105.000000,110.100000,flow_00.1,-1031.000000
8,4.5,flow_00.0,90.552513,86.461988,30.000000,1000.000000,NaN,6.470393e-10,6.470393e-10,6.470393e-10,6.470393e-10,-6.291927e-10,0,486885259,2,120.000000,125.100000,flow_00.1,-1031.000000
9,5.0,flow_00.0,101.628332,96.579277,30.000000,1000.000000,NaN,6.889853e-10,6.889853e-10,6.889853e-10,6.889853e-10,-5.248637e-10,0,486885259,2,135.000000,140.100000,flow_00.1,-1031.000000


In [1]:
import MarocEnv as MarocEnv

In [11]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env

from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter
from flow.networks import Network
from flow.core.params import InFlows
from flow.envs import MarocEnv as MarocEnv


# time horizon of a single rollout
HORIZON = 1500
# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2
ray.init(num_cpus=N_CPUS)

2021-10-12 01:39:01,898	INFO resource_spec.py:212 -- Starting Ray with 1.27 GiB memory available for workers and up to 0.65 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2021-10-12 01:39:03,059	INFO services.py:1165 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.1.4',
 'raylet_ip_address': '192.168.1.4',
 'redis_address': '192.168.1.4:6379',
 'object_store_address': 'tcp://127.0.0.1:61705',
 'raylet_socket_name': 'tcp://127.0.0.1:59627',
 'webui_url': 'localhost:8265',
 'session_dir': 'C:\\Users\\rhanizar\\AppData\\Local\\Temp\\ray\\session_2021-10-12_01-39-01_823467_19564'}

In [1]:
#from flow.envs import AccelEnv as MarocEnv
#Edges (road segments)
EDGES_DISTRIBUTION = [
    "486885259",
    "154190805",
]
#Road segments positions
class MarocNetwork(Network):

    def specify_routes(self, net_params1):
        return {
            "486885259": [
            "486885259",],

            "154190805" : [
            "154190805",],}


vehicles = VehicleParams()

#Human driving vehicles (initially 3)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
            color="white")

#flow of 3000 vehicles per hour for the segment studied (aller)
inflow = InFlows()
inflow.add(
   veh_type = "human",
   edge="486885259",
   vehs_per_hour=3000,
    depart_lane="random",
    depart_speed = "max",
    begin = 1,
    color="green")



# NETWORK
network = MarocNetwork
# NETWORK PARAMS
additional_net_params = ADDITIONAL_NET_PARAMS.copy()
#33 m/s = 120 Km/s
additional_net_params['speed_limit'] = 33
additional_net_params['highway_lanes'] = 3

#absolute Path to network file 
abs_path = 'C://PHD/VSL/RL-VSL/Project SUMO-VSL/flow-master/tutorials/networks/maroc.net.xml'
net_params1 = NetParams(inflows=inflow, 
                        additional_params=additional_net_params, template=abs_path)


flow_params = dict(
    # name of the experiment
    exp_tag="agent_VSL",

    # name of the flow environment the experiment is running on
    env_name=MarocEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=True,
        emission_path='data'
    ),
    
     # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=750,
        clip_actions=False,
        additional_params={
            "target_velocity": 33,
            "max_accel": 1,
            "max_decel": 1,
        },
    ),
    
    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params1,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=InitialConfig(
        bunching=20,
        edges_distribution=EDGES_DISTRIBUTION
    ),
)

In [9]:

def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config


alg_run, gym_name, config = setup_exps()
ray.shutdown()
ray.init(num_cpus=N_CPUS + 1, ignore_reinit_error=True)


print("Dashboard URL: http://{}".format(ray.get_webui_url()))

trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 1,
        },
    }
})

NameError: name 'get_agent_class' is not defined

In [12]:
# The algorithm or model to train. This may refer to "
#      "the name of a built-on algorithm (e.g. RLLib's DQN "
#      "or PPO), or a user-defined trainable function or "
#      "class registered in the tune registry.")
alg_run = "PPO"

agent_cls = get_agent_class(alg_run)
config = agent_cls._default_config.copy()
config["num_workers"] = N_CPUS - 1  # number of parallel workers
config["train_batch_size"] = HORIZON * N_ROLLOUTS  # batch size
config["gamma"] = 0.999  # discount rate
config["model"].update({"fcnet_hiddens": [16, 16]})  # size of hidden layers in network
config["use_gae"] = True  # using generalized advantage estimation
config["lambda"] = 0.97  
config["sgd_minibatch_size"] = min(16 * 1024, config["train_batch_size"])  # stochastic gradient descent
config["kl_target"] = 0.02  # target KL divergence
config["num_sgd_iter"] = 10  # number of SGD iterations
config["horizon"] = HORIZON  # rollout horizon

# save the flow params for replay
flow_json = json.dumps(flow_params, cls=FlowParamsEncoder, sort_keys=True,
                       indent=4)  # generating a string version of flow_params
config['env_config']['flow_params'] = flow_json  # adding the flow_params to config dict
config['env_config']['run'] = alg_run

# Call the utility function make_create_env to be able to 
# register the Flow env for this experiment
create_env, gym_name = make_create_env(params=flow_params, version=0)

# Register as rllib env with Gym
register_env(gym_name, create_env)
ray.shutdown()
ray.init(num_cpus=N_CPUS + 1, ignore_reinit_error=True)


print("Dashboard URL: http://{}".format(ray.get_webui_url()))

2021-10-12 01:39:11,485	INFO resource_spec.py:212 -- Starting Ray with 1.22 GiB memory available for workers and up to 0.63 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2021-10-12 01:39:12,288	INFO services.py:1165 -- View the Ray dashboard at localhost:8265


Dashboard URL: http://localhost:8265


2021-10-12 01:39:19,392	WARNING worker.py:1047 -- The dashboard on node POSTE1 failed with the following error:
Traceback (most recent call last):
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\aiohttp\web_urldispatcher.py", line 553, in __init__
    raise ValueError("Not a directory")
ValueError: Not a directory

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\dashboard/dashboard.py", line 960, in <module>
    metrics_export_address=metrics_export_address)
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\dashboard/dashboard.py", line 514, in __init__
    setup_speedscope_dir(self.app, build_dir)
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\dashboard/dashboard.py", line 423, in setup_speedscope_dir
    app.router.add_static("/speedscope", speedscope_dir)
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\s

In [13]:
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 1,  # number of iterations between checkpoints
        "checkpoint_at_end": True,  # generate a checkpoint at the end
        "max_failures": 2,
        "stop": {  # stopping conditions
            "training_iteration": 1,  # number of iterations to stop after
        },
    },
})

2021-10-12 01:39:45,119	ERROR syncer.py:46 -- Log sync requires rsync to be installed.


Trial name,status,loc
PPO_WaveAttenuationPOEnv-v0_e54e4_00000,RUNNING,


2021-10-12 01:39:52,533	ERROR trial_runner.py:520 -- Trial PPO_WaveAttenuationPOEnv-v0_e54e4_00000: Error processing event.
Traceback (most recent call last):
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\tune\trial_runner.py", line 468, in _process_trial
    result = self.trial_executor.fetch_result(trial)
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\tune\ray_trial_executor.py", line 430, in fetch_result
    result = ray.get(trial_future[0], DEFAULT_GET_TIMEOUT)
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\worker.py", line 1474, in get
    raise value.as_instanceof_cause()
ray.exceptions.RayTaskError(UnregisteredEnv): ray::PPO.train() (pid=15252, ip=192.168.1.4)
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\gym\envs\registration.py", line 132, in spec
    return self.env_specs[id]
KeyError: 'WaveAttenuationPOEnv-v0'

During handling of the above exception, another exception occurred:

ray::PPO.train(

Trial name,status,loc
PPO_WaveAttenuationPOEnv-v0_e54e4_00000,RUNNING,
Trial name,# failures,error file
PPO_WaveAttenuationPOEnv-v0_e54e4_00000,1,C:\Users\rhanizar\ray_results\agent_VSL\PPO_WaveAttenuationPOEnv-v0_0_2021-10-12_01-39-44fuso3s70\error.txt


(pid=15252) 2021-10-12 01:39:52,510	INFO trainer.py:585 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=15252) 2021-10-12 01:39:52,510	INFO trainer.py:612 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


...

(pid=18160) 2021-10-12 01:40:12,222	INFO trainer.py:585 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
(pid=18160) 2021-10-12 01:40:12,231	INFO trainer.py:612 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


Trial name,status,loc
PPO_WaveAttenuationPOEnv-v0_e54e4_00000,ERROR,
Trial name,# failures,error file
PPO_WaveAttenuationPOEnv-v0_e54e4_00000,3,C:\Users\rhanizar\ray_results\agent_VSL\PPO_WaveAttenuationPOEnv-v0_0_2021-10-12_01-39-44fuso3s70\error.txt


2021-10-12 01:40:12,281	ERROR tune.py:334 -- Trial Runner checkpointing failed.
Traceback (most recent call last):
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\tune\tune.py", line 332, in run
    runner.checkpoint(force=True)
  File "c:\users\rhanizar\anaconda3\envs\flow\lib\site-packages\ray\tune\trial_runner.py", line 279, in checkpoint
    os.rename(tmp_file_name, self.checkpoint_file)
FileExistsError: [WinError 183] Impossible de créer un fichier déjà existant: 'C:\\Users\\rhanizar\\ray_results\\agent_VSL\\.tmp_checkpoint' -> 'C:\\Users\\rhanizar\\ray_results\\agent_VSL\\experiment_state-2021-10-12_01-39-44.json'


Trial name,status,loc
PPO_WaveAttenuationPOEnv-v0_e54e4_00000,ERROR,
Trial name,# failures,error file
PPO_WaveAttenuationPOEnv-v0_e54e4_00000,3,C:\Users\rhanizar\ray_results\agent_VSL\PPO_WaveAttenuationPOEnv-v0_0_2021-10-12_01-39-44fuso3s70\error.txt


TuneError: ('Trials did not complete', [PPO_WaveAttenuationPOEnv-v0_e54e4_00000])